In [16]:
from google.colab import files
uploaded = files.upload()

Saving victorian_road_crash_data.csv to victorian_road_crash_data (1).csv


In [17]:
import pandas as pd

df = pd.read_csv('victorian_road_crash_data.csv')
print(df.shape)
df.head()
df.info()


(184605, 52)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184605 entries, 0 to 184604
Data columns (total 52 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   ACCIDENT_NO             184605 non-null  object 
 1   ACCIDENT_DATE           184605 non-null  object 
 2   ACCIDENT_TIME           184605 non-null  object 
 3   ACCIDENT_TYPE           184605 non-null  object 
 4   DAY_OF_WEEK             184605 non-null  object 
 5   DCA_CODE                184605 non-null  int64  
 6   DCA_CODE_DESCRIPTION    184605 non-null  object 
 7   LIGHT_CONDITION         184605 non-null  object 
 8   POLICE_ATTEND           184605 non-null  object 
 9   ROAD_GEOMETRY           184605 non-null  object 
 10  SEVERITY                184605 non-null  object 
 11  SPEED_ZONE              184605 non-null  object 
 12  RUN_OFFROAD             184605 non-null  object 
 13  ROAD_NAME               184361 non-null  object 
 14  ROAD_TY

In [18]:
columns_to_keep = ['ACCIDENT_NO', 'ACCIDENT_DATE', 'DAY_OF_WEEK', 'ACCIDENT_TIME','ACCIDENT_TYPE','SPEED_ZONE', 'LIGHT_CONDITION','ROAD_GEOMETRY','ROAD_NAME','ROAD_TYPE', 'LATITUDE', 'LONGITUDE', 'PEDESTRIAN']
df = df[columns_to_keep]
df.head()

,ACCIDENT_NO,ACCIDENT_DATE,DAY_OF_WEEK,ACCIDENT_TIME,ACCIDENT_TYPE,SPEED_ZONE,LIGHT_CONDITION,ROAD_GEOMETRY,ROAD_NAME,ROAD_TYPE,LATITUDE,LONGITUDE,PEDESTRIAN
0,T20230013207,2023-06-06,Tuesday,22:14:00,Collision with vehicle,50 km/hr,Dark Street lights on,Not at intersection,CUMBERLAND,ROAD,-37.723442,144.93985,0
1,T20240014902,2024-04-19,Friday,02:10:00,Collision with vehicle,40 km/hr,Dark Street lights on,Not at intersection,CHAPEL,STREET,-37.850610,144.99355,0
2,T20160009452,2016-04-30,Saturday,23:50:00,Collision with vehicle,100 km/hr,Dark No street lights,Not at intersection,BRANDY CREEK,ROAD,-38.111640,145.92566,0
3,T20230001223,2023-01-17,Tuesday,15:56:00,Collision with vehicle,100 km/hr,Day,Private property,MIDLAND,HIGHWAY,-36.409386,145.25609,0
4,T20220001324,2022-01-21,Friday,22:45:00,Fall from or in moving vehicle,50 km/hr,Dark Street lights on,Cross intersection,URQUHART,STREET,-37.065030,144.22215,0


In [19]:
#Find how many nulls in critical columns
print("Nulls in critical columns:")
print(df[['LATITUDE', 'LONGITUDE', 'ACCIDENT_DATE']].isnull().sum())

#Print rows where any of those columns are missing
missing_rows = df[df[['LATITUDE', 'LONGITUDE', 'ACCIDENT_DATE']].isnull().any(axis=1)]
print("\nRows with missing LATITUDE, LONGITUDE or ACCIDENT_DATE:")
print(missing_rows)

df.dropna(subset=['LATITUDE', 'LONGITUDE', 'ACCIDENT_DATE'], inplace=True)

Nulls in critical columns:
LATITUDE         85
LONGITUDE        85
ACCIDENT_DATE     0
dtype: int64

Rows with missing LATITUDE, LONGITUDE or ACCIDENT_DATE:
         ACCIDENT_NO ACCIDENT_DATE DAY_OF_WEEK     ACCIDENT_TIME  \
45215   T20180021544    2018-11-05      Monday  13:00:00.0000000   
45216   T20130002446    2012-08-20      Monday  15:30:00.0000000   
45217   T20180023958    2018-05-14      Monday  20:30:00.0000000   
45218   T20120005932    2012-03-10    Saturday  14:00:00.0000000   
45219   T20200004272    2020-01-13      Monday  15:05:00.0000000   
...              ...           ...         ...               ...   
184600  T20120020447    2012-08-12      Sunday  14:00:00.0000000   
184601  T20190023830    2019-11-23    Saturday  17:15:00.0000000   
184602  T20170015401    2017-08-09   Wednesday  12:00:00.0000000   
184603  T20120013446    2012-06-09    Saturday  01:25:00.0000000   
184604  T20120006036    2012-03-08    Thursday  18:26:00.0000000   

                          

In [20]:
#Rounding Lat Long Values
df['LATITUDE'] = df['LATITUDE'].round(2)
df['LONGITUDE'] = df['LONGITUDE'].round(2)

In [21]:
geometry_map = {
    'Cross intersection': 1,
    'T intersection': 2,
    'Y intersection': 3,
    'Multiple intersections': 4,
    'Not at intersection': 5,
    'Dead end': 6,
    'Road closure': 7,
    'Private property': 8,
    'Unknown': 9
}
df['intersection_type'] = df['ROAD_GEOMETRY'].map(geometry_map)

# 1 = Yes (if code is in 1–4), 0 = No (if code is 5+)
df['is_intersection'] = df['intersection_type'].apply(lambda x: 1 if x in [1,2,3,4] else 0)
# Complex = 1 if multiple/Y; otherwise 0
df['complex_intersection'] = df['intersection_type'].apply(lambda x: 1 if x in [3,4] else 0)
print(df.head())

    ACCIDENT_NO ACCIDENT_DATE DAY_OF_WEEK ACCIDENT_TIME  \
0  T20230013207    2023-06-06     Tuesday      22:14:00   
1  T20240014902    2024-04-19      Friday      02:10:00   
2  T20160009452    2016-04-30    Saturday      23:50:00   
3  T20230001223    2023-01-17     Tuesday      15:56:00   
4  T20220001324    2022-01-21      Friday      22:45:00   

                    ACCIDENT_TYPE SPEED_ZONE        LIGHT_CONDITION  \
0          Collision with vehicle   50 km/hr  Dark Street lights on   
1          Collision with vehicle   40 km/hr  Dark Street lights on   
2          Collision with vehicle  100 km/hr  Dark No street lights   
3          Collision with vehicle  100 km/hr                    Day   
4  Fall from or in moving vehicle   50 km/hr  Dark Street lights on   

         ROAD_GEOMETRY     ROAD_NAME ROAD_TYPE  LATITUDE  LONGITUDE  \
0  Not at intersection    CUMBERLAND      ROAD    -37.72     144.94   
1  Not at intersection        CHAPEL    STREET    -37.85     144.99   
2  No

In [22]:
#Conversion
df['Accident_Date'] = pd.to_datetime(df['ACCIDENT_DATE'], errors='coerce')
df['Accident_Time'] = pd.to_datetime(df['ACCIDENT_TIME'], format='%H:%M:%S', errors='coerce').dt.time

# Extract features
df['Year'] = df['Accident_Date'].dt.year
df['Month'] = df['Accident_Date'].dt.month
print(df.head())

    ACCIDENT_NO ACCIDENT_DATE DAY_OF_WEEK ACCIDENT_TIME  \
0  T20230013207    2023-06-06     Tuesday      22:14:00   
1  T20240014902    2024-04-19      Friday      02:10:00   
2  T20160009452    2016-04-30    Saturday      23:50:00   
3  T20230001223    2023-01-17     Tuesday      15:56:00   
4  T20220001324    2022-01-21      Friday      22:45:00   

                    ACCIDENT_TYPE SPEED_ZONE        LIGHT_CONDITION  \
0          Collision with vehicle   50 km/hr  Dark Street lights on   
1          Collision with vehicle   40 km/hr  Dark Street lights on   
2          Collision with vehicle  100 km/hr  Dark No street lights   
3          Collision with vehicle  100 km/hr                    Day   
4  Fall from or in moving vehicle   50 km/hr  Dark Street lights on   

         ROAD_GEOMETRY     ROAD_NAME ROAD_TYPE  LATITUDE  LONGITUDE  \
0  Not at intersection    CUMBERLAND      ROAD    -37.72     144.94   
1  Not at intersection        CHAPEL    STREET    -37.85     144.99   
2  No

In [23]:
#Delete duplicate columns
df.drop(columns=['ACCIDENT_DATE', 'ACCIDENT_TIME'], inplace=True)
print(df.head())

    ACCIDENT_NO DAY_OF_WEEK                   ACCIDENT_TYPE SPEED_ZONE  \
0  T20230013207     Tuesday          Collision with vehicle   50 km/hr   
1  T20240014902      Friday          Collision with vehicle   40 km/hr   
2  T20160009452    Saturday          Collision with vehicle  100 km/hr   
3  T20230001223     Tuesday          Collision with vehicle  100 km/hr   
4  T20220001324      Friday  Fall from or in moving vehicle   50 km/hr   

         LIGHT_CONDITION        ROAD_GEOMETRY     ROAD_NAME ROAD_TYPE  \
0  Dark Street lights on  Not at intersection    CUMBERLAND      ROAD   
1  Dark Street lights on  Not at intersection        CHAPEL    STREET   
2  Dark No street lights  Not at intersection  BRANDY CREEK      ROAD   
3                    Day     Private property       MIDLAND   HIGHWAY   
4  Dark Street lights on   Cross intersection      URQUHART    STREET   

   LATITUDE  LONGITUDE  PEDESTRIAN  intersection_type  is_intersection  \
0    -37.72     144.94           0        

In [24]:
#Filter out Pedestrian Data
df = df[df['ACCIDENT_TYPE'].str.lower() == 'struck pedestrian']
print(df.head())

     ACCIDENT_NO DAY_OF_WEEK      ACCIDENT_TYPE SPEED_ZONE LIGHT_CONDITION  \
6   T20150009796    Saturday  Struck Pedestrian   40 km/hr       Dusk/Dawn   
8   T20160000309      Monday  Struck Pedestrian   40 km/hr             Day   
28  T20240018569      Monday  Struck Pedestrian   60 km/hr             Day   
37  T20180000465     Tuesday  Struck Pedestrian   60 km/hr             Day   
61  T20140004479     Tuesday  Struck Pedestrian   60 km/hr       Dusk/Dawn   

          ROAD_GEOMETRY              ROAD_NAME ROAD_TYPE  LATITUDE  LONGITUDE  \
6    Cross intersection                COLLINS    STREET    -37.82     144.97   
8    Cross intersection                   KING    STREET    -37.82     144.96   
28  Not at intersection              CAMBRIDGE      ROAD    -37.79     145.31   
37  Not at intersection  BRIGHTON ROAD SERVICE      ROAD    -37.88     145.00   
61  Not at intersection                 BOURKE      ROAD    -37.94     145.11   

    PEDESTRIAN  intersection_type  is_inters

In [25]:
#Rename columns
df.rename(columns={
    'ACCIDENT_NO': 'Accident_No',
    'DAY_OF_WEEK': 'Day_of_Week',
    'ACCIDENT_TYPE': 'Accident_Type',
    'SPEED_ZONE':'Speed_Zone',
    'LIGHT_CONDITION': 'Light_Condition',
    'ROAD_GEOMETRY': 'Road_Geometry',
    'ROAD_NAME': 'Road_Name',
    'ROAD_TYPE': 'Road_Type',
    'LATITUDE': 'Latitude',
    'LONGITUDE': 'Longitude',
    'PEDESTRIAN': 'Pedestrian',
}, inplace=True)
print(df.head())

     Accident_No Day_of_Week      Accident_Type Speed_Zone Light_Condition  \
6   T20150009796    Saturday  Struck Pedestrian   40 km/hr       Dusk/Dawn   
8   T20160000309      Monday  Struck Pedestrian   40 km/hr             Day   
28  T20240018569      Monday  Struck Pedestrian   60 km/hr             Day   
37  T20180000465     Tuesday  Struck Pedestrian   60 km/hr             Day   
61  T20140004479     Tuesday  Struck Pedestrian   60 km/hr       Dusk/Dawn   

          Road_Geometry              Road_Name Road_Type  Latitude  Longitude  \
6    Cross intersection                COLLINS    STREET    -37.82     144.97   
8    Cross intersection                   KING    STREET    -37.82     144.96   
28  Not at intersection              CAMBRIDGE      ROAD    -37.79     145.31   
37  Not at intersection  BRIGHTON ROAD SERVICE      ROAD    -37.88     145.00   
61  Not at intersection                 BOURKE      ROAD    -37.94     145.11   

    Pedestrian  intersection_type  is_inters

In [26]:
#Filter out Data from 01/01/2023
df = df[df['Accident_Date'] >= pd.to_datetime('2023-01-01')]
print(df.head())

      Accident_No Day_of_Week      Accident_Type Speed_Zone  \
28   T20240018569      Monday  Struck Pedestrian   60 km/hr   
71   T20230000174     Tuesday  Struck Pedestrian   60 km/hr   
83   T20240019455    Thursday  Struck Pedestrian   60 km/hr   
168  T20240018917      Friday  Struck Pedestrian   40 km/hr   
217  T20230016318    Thursday  Struck Pedestrian  Not known   

           Light_Condition        Road_Geometry  Road_Name Road_Type  \
28                     Day  Not at intersection  CAMBRIDGE      ROAD   
71   Dark Street lights on  Not at intersection      BARRY      ROAD   
83                     Day   Cross intersection    MALVERN      ROAD   
168  Dark Street lights on   Cross intersection     REGENT       NaN   
217                   Unk.  Not at intersection      PETER     COURT   

     Latitude  Longitude  Pedestrian  intersection_type  is_intersection  \
28     -37.79     145.31           1                5.0                0   
71     -37.67     144.95           1

In [27]:
#Create column to determine Weekday or Weekend
df.loc[:, 'Day_Type'] = df['Day_of_Week'].str.strip().str.capitalize().apply(
    lambda x: 'Weekend' if x in ['Saturday', 'Sunday'] else 'Weekday'
)
print(df.head())

      Accident_No Day_of_Week      Accident_Type Speed_Zone  \
28   T20240018569      Monday  Struck Pedestrian   60 km/hr   
71   T20230000174     Tuesday  Struck Pedestrian   60 km/hr   
83   T20240019455    Thursday  Struck Pedestrian   60 km/hr   
168  T20240018917      Friday  Struck Pedestrian   40 km/hr   
217  T20230016318    Thursday  Struck Pedestrian  Not known   

           Light_Condition        Road_Geometry  Road_Name Road_Type  \
28                     Day  Not at intersection  CAMBRIDGE      ROAD   
71   Dark Street lights on  Not at intersection      BARRY      ROAD   
83                     Day   Cross intersection    MALVERN      ROAD   
168  Dark Street lights on   Cross intersection     REGENT       NaN   
217                   Unk.  Not at intersection      PETER     COURT   

     Latitude  Longitude  Pedestrian  intersection_type  is_intersection  \
28     -37.79     145.31           1                5.0                0   
71     -37.67     144.95           1

In [28]:
# Group by lat/lon and count number of pedestrian crashes
ped_counts = df.groupby(['Latitude', 'Longitude']).size().reset_index(name='total_pedestrian_crashes')

# Merge crash counts back into main DataFrame
df = df.merge(ped_counts, on=['Latitude', 'Longitude'], how='left')

# Replace NaNs (non-pedestrian areas) with 0
df['total_pedestrian_crashes'] = df['total_pedestrian_crashes'].fillna(0).astype(int)

df[['Latitude', 'Longitude', 'Accident_Type', 'total_pedestrian_crashes']].head()



,Latitude,Longitude,Accident_Type,total_pedestrian_crashes
0,-37.79,145.31,Struck Pedestrian,4
1,-37.67,144.95,Struck Pedestrian,4
2,-37.86,145.06,Struck Pedestrian,1
3,-38.17,144.34,Struck Pedestrian,4
4,-37.97,145.22,Struck Pedestrian,3


In [30]:
#Export to csv
column_order = [
    'Accident_No', 'Accident_Date', 'Accident_Time', 'Day_of_Week', 'Day_Type',
    'Accident_Type','Speed_Zone', 'Light_Condition', 'Road_Geometry','is_intersection','complex_intersection', 'Road_Name', 'Road_Type',
    'Latitude', 'Longitude', 'Pedestrian', 'Year', 'Month','total_pedestrian_crashes'
]
df = df[column_order]
df.to_csv("cleaned_crash_data.csv", index=False)


In [31]:
df.to_csv("cleaned_crash_data.csv", index=False)

from google.colab import files
files.download("cleaned_crash_data.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>